# Imports

In [1]:
import src.acquire

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# Problem Statement | Background

# Acquire

In [2]:
df = src.acquire.get_zillow_data()

CSV previously generated at `data/raw/zillow_unprocessed.csv`. Reading in that csv as a DataFrame


## Where the Data is Coming From

## How the Data is Being Filtered

# Wrangle

## Dtypes

> Write a function that takes in a dataframe and a list of column names and returns the dataframe with the datatypes of those columns changed to a non-numeric type.

> Use this function to appropriately transform any numeric columns that should not be treated as numbers.

## Missing Values

>  Impute the values in land square feet.

> For land square feet, the goal is to impute the missing values by creating a linear model where landtaxvaluedollarcnt is the x-variable and the output/y-variable is the estimated land square feet.

> We'll then use this model to make predictions and fill in the missing values.

> Write a function that accepts the zillow data frame and returns the data frame with the missing values filled in.

# Exploration

# Modeling

# Evaluation